## 2 Private Training
Student Name & Number =Sara Rezanezhad / 99101643
Student Name & Number =Kimia Fakheri / *******


## Import necessary packages

In [1]:
!pip install -U scikit-learn
!pip install -U scikit-learn tensorflow matplotlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 45.0 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 MB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 50.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 55.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 53.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 60.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 47.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 903.3 kB/s eta 0:00:00
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.2.0
    Uninstalling ml-dtypes-0.2.0:
      Successfully uninstalled ml-dtypes-0.2.0
  Attempting unins

In [2]:
import tensorflow as tf
import tensorflow.keras
from tensorflow.keras.utils import to_categorical
import numpy as np
from sklearn.utils import resample
from sklearn.metrics import ConfusionMatrixDisplay
import matplotlib.pyplot as plt
from matplotlib import pyplot

## Needed Functions

In [3]:
def load_fashion_mnist_dataset():
    from keras.datasets import fashion_mnist

    # load dataset
    (trainX, trainY), (testX, testY) = fashion_mnist.load_data()

    # reshape dataset to have a single channel
    trainX = trainX.reshape((trainX.shape[0], 28, 28, 1))
    testX = testX.reshape((testX.shape[0], 28, 28, 1))

    # one hot encode target values
    trainY = to_categorical(trainY)
    testY = to_categorical(testY)

    return (trainX, trainY), (testX, testY)


num_epochs = 5
VERBOSE = 2


def sample_data(train_data, test_data, num_sets):
    (x_train, y_train), (x_test, y_test) = train_data, test_data
    x_train2, y_train2 = [], []
    x_test2, y_test2 = [], []
    for i in range(num_sets):
        x_temp, y_temp = resample(x_train, y_train, n_samples=3000, random_state=0)
        x_train2.append(x_temp)
        y_train2.append(y_temp)
        x_temp, y_temp = resample(x_test, y_test, n_samples=2000, random_state=0)
        x_test2.append(x_temp)
        y_test2.append(y_temp)
    return (x_train2, y_train2), (x_test2, y_test2)

def Attack_dataset(models, train_data, test_data, num_models, data_size):
    # generate dataset for the attack model
    (x_train, y_train), (x_test, y_test) = train_data, test_data
    # set number of classes for the attack model
    num_classes = 10
    x_data, y_data = [[] for _ in range(num_classes)], [[] for _ in range(num_classes)]
    for i in range(num_models):
        #  Data points that are part of the target model's training dataset.
        x_temp, y_temp = resample(x_train[i], y_train[i], n_samples=data_size, random_state=0)
        for j in range(data_size):
            y_idx = np.argmax(y_temp[j])
            x_data[y_idx].append(models[i].predict(x_temp[j:j+1], verbose=0)[0])
            y_data[y_idx].append(1) #membership = 1(true)

        # Data points not used during target model training.
        x_temp, y_temp = resample(x_test[i], y_test[i], n_samples=data_size, random_state=0)
        for j in range(data_size):
            y_idx = np.argmax(y_temp[j])
            x_data[y_idx].append(models[i].predict(x_temp[j:j+1], verbose=0)[0])
            y_data[y_idx].append(0) #membership = 0(fulse)

    return x_data, y_data

#Fully Connected Neural Network (FCNN) model for the Fashion-MNIST dataset:
def FCNN():
    model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
    ])
    return model

def training_models(keras_model, train_data, test_data, num_models):
    (x_train, y_train), (x_test, y_test) = train_data, test_data
    models = []
    for i in range(num_models):
        models.append(tf.keras.models.clone_model(keras_model))
        rms = tf.keras.optimizers.RMSprop(learning_rate=0.001, decay=1e-7)
        models[i].compile(loss='categorical_crossentropy', optimizer=rms, metrics=['accuracy'])
        models[i].fit(x_train[i], y_train[i], batch_size=32, epochs=num_epochs, verbose=VERBOSE, shuffle=True)
        score = models[i].evaluate(x_test[i], y_test[i], verbose=VERBOSE)
        print('\n', 'Model ', i, ' test accuracy:', score[1])
    return models


#trains Support Vector Machine (SVM) models on the attack dataset:
def training_svm_models(train_data, test_data, num_models=1):
    from sklearn import svm
    (x_train, y_train), (x_test, y_test) = train_data, test_data
    models = []
    for i in range(num_models):
        print('Training svm model : ', i)
        models.append(svm.SVC(gamma='scale', kernel='linear', verbose=VERBOSE))
        models[i].fit(x_train[i], y_train[i])
        score = models[i].score(x_test[i], y_test[i])
        print('SVM model ', i, 'score : ', score)
    return models

def Attack_dataset(models, train_data, test_data, num_models, data_size):
    # generate dataset for the attack model
    (x_train, y_train), (x_test, y_test) = train_data, test_data
    # set number of classes for the attack model
    num_classes = 10
    x_data, y_data = [[] for _ in range(num_classes)], [[] for _ in range(num_classes)]
    for i in range(num_models):
        #  Data points that are part of the target model's training dataset.
        x_temp, y_temp = resample(x_train[i], y_train[i], n_samples=data_size, random_state=0)
        for j in range(data_size):
            y_idx = np.argmax(y_temp[j])
            x_data[y_idx].append(models[i].predict(x_temp[j:j+1], verbose=0)[0])
            y_data[y_idx].append(1) #membership = 1(true)

        # Data points not used during target model training.
        x_temp, y_temp = resample(x_test[i], y_test[i], n_samples=data_size, random_state=0)
        for j in range(data_size):
            y_idx = np.argmax(y_temp[j])
            x_data[y_idx].append(models[i].predict(x_temp[j:j+1], verbose=0)[0])
            y_data[y_idx].append(0) #membership = 0(fulse)

    return x_data, y_data




## Q 4 , 6 , 7 For Baseline Model

In [4]:
# load the pre-shuffled train and test data
(x_train, y_train), (x_test, y_test) = load_fashion_mnist_dataset()

# split the data for each model
target_train = (x_train[:3000*1], y_train[:3000*1])
target_test = (x_test[:2000*1], y_test[:2000*1])
target_train_data, target_test_data = sample_data(target_train, target_test, 1)

shadow_train = (x_train[3000*1:], y_train[3000*1:])
shadow_test = (x_test[2000*1:], y_test[2000*1:])
shadow_train_data, shadow_test_data = sample_data(shadow_train, shadow_test, 2)

cnn_model = FCNN()

# Q4
target_models = training_models(cnn_model, target_train_data, target_test_data, 1)
# Q6
shadow_models = training_models(cnn_model, shadow_train_data, shadow_test_data, 2)

# get train data for the attack model
attack_train = Attack_dataset(shadow_models, shadow_train_data, shadow_test_data, 2, 2000)
# get test data for the attack model
attack_test = Attack_dataset(target_models, target_train_data, target_test_data, 1, 2000)
#Q7
# training the attack model
attack_model = training_svm_models(attack_train, attack_test)

29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 1us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


/usr/local/lib/python3.10/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
/usr/local/lib/python3.10/dist-packages/keras/src/optimizers/base_optimizer.py:33: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


Epoch 1/5
94/94 - 1s - 11ms/step - accuracy: 0.6143 - loss: 38.9741
Epoch 2/5
94/94 - 0s - 3ms/step - accuracy: 0.7207 - loss: 9.8141
Epoch 3/5
94/94 - 1s - 6ms/step - accuracy: 0.7007 - loss: 2.7827
Epoch 4/5
94/94 - 0s - 3ms/step - accuracy: 0.7357 - loss: 1.5999
Epoch 5/5
94/94 - 0s - 3ms/step - accuracy: 0.7437 - loss: 1.3493
63/63 - 0s - 4ms/step - accuracy: 0.6745 - loss: 1.9283

 Model  0  test accuracy: 0.6744999885559082
Epoch 1/5
94/94 - 1s - 11ms/step - accuracy: 0.5937 - loss: 34.0924
Epoch 2/5
94/94 - 0s - 3ms/step - accuracy: 0.6920 - loss: 9.5693
Epoch 3/5
94/94 - 0s - 3ms/step - accuracy: 0.6850 - loss: 3.5583
Epoch 4/5
94/94 - 1s - 8ms/step - accuracy: 0.6410 - loss: 1.5153
Epoch 5/5
94/94 - 1s - 6ms/step - accuracy: 0.6870 - loss: 1.3075
63/63 - 0s - 7ms/step - accuracy: 0.6815 - loss: 1.5208

 Model  0  test accuracy: 0.6815000176429749
Epoch 1/5
94/94 - 1s - 11ms/step - accuracy: 0.5713 - loss: 52.9235
Epoch 2/5
94/94 - 0s - 3ms/step - accuracy: 0.7090 - loss: 14.61